# Working with IPython and dask.distributed

[dask.distributed](https://distributed.readthedocs.io) is a cool library for doing distributed execution. You should check it out, if you haven't already.

In many cases, dask.distributed should replace using IPython Parallel if you primarily use the LoadBalancedView.

However, you may already have infrastructure for deploying and managing IPython engines,
and IPython Parallel's interactive debugging features can still be useful.

Any IPython cluster can *become* a dask clsuter at any time,
and be used simultaneously via both APIs.

You can turn your IPython cluster into a distributed cluster by calling `Client.become_dask()`:

In [1]:
import ipyparallel as ipp

cluster = ipp.Cluster()
n = 4
await cluster.start_cluster(n)
rc = await cluster.connect_client()
rc.wait_for_engines(n)

Using existing profile dir: '/Users/minrk/.ipython/profile_default'
Starting 4 engines with <class 'ipyparallel.cluster.launcher.LocalEngineSetLauncher'>


  0%|          | 0/4 [00:00<?, ?engine/s]

In [2]:
dask_client = rc.become_dask(ncores=1)
dask_client

<Client: 'tcp://192.168.1.31:53264' processes=1 threads=1, memory=4.00 GiB>

This will:

1. start a Scheduler on the Hub
2. start a Worker on each engine
3. return an Executor, the distributed client API

By default, distributed Workers will use threads to run on all cores of a machine. 
In this case, since I already have one *engine* per core,
I tell distributed to run one core per Worker with `ncores=1`.

We can now use our IPython cluster with distributed:

In [3]:
from distributed import progress


def square(x):
    return x ** 2


def neg(x):
    return -x


A = dask_client.map(square, range(1000))
B = dask_client.map(neg, A)
total = dask_client.submit(sum, B)
progress(total)

VBox()

In [4]:
total.result()

-332833500

I could also let distributed do its multithreading thing, and run one multi-threaded Worker per engine.

First, I need to get a mapping of one engine per host:

In [5]:
import socket

engine_hosts = rc[:].apply_async(socket.gethostname).get_dict()
engine_hosts

{0: 'touchy', 1: 'touchy', 2: 'touchy', 3: 'touchy'}

I can reverse this mapping, to get a list of engines on each host:

In [6]:
host_engines = {}
for engine_id, host in engine_hosts.items():
    if host not in host_engines:
        host_engines[host] = []
    host_engines[host].append(engine_id)

host_engines

{'touchy': [0, 1, 2, 3]}

Now I can get one engine per host:

In [7]:
one_engine_per_host = [engines[0] for engines in host_engines.values()]
one_engine_per_host

[0]

*Here's a concise, but more opaque version that does the same thing:*

In [8]:
one_engine_per_host = list({host: eid for eid, host in engine_hosts.items()}.values())
one_engine_per_host

[3]

I can now stop the first distributed cluster, and start a new one on just these engines, letting distributed allocate threads:

In [9]:
rc.stop_distributed()

dask_client = rc.become_dask(one_engine_per_host)
dask_client

<Client: 'tcp://192.168.1.31:54313' processes=1 threads=1, memory=4.00 GiB>

And submit the same tasks again:

In [10]:
A = dask_client.map(square, range(100))
B = dask_client.map(neg, A)
total = dask_client.submit(sum, B)
progress(total)

VBox()

## Debugging distributed with IPython

In [11]:
rc.stop_distributed()

dask_client = rc.become_dask(one_engine_per_host)
dask_client

Connection method: Direct,
Dashboard: http://192.168.1.31:54467/status,
Comm: tcp://192.168.1.31:54478,Workers: 0
Dashboard: http://192.168.1.31:54467/status,Total threads: 0
Started: Just now,Total memory: 0 B


Let's set the %px magics to only run on our one engine per host:

In [12]:
view = rc[one_engine_per_host]
view.block = True
view.activate()

Let's submit some work that's going to fail somewhere in the middle:

In [13]:
from IPython.display import display
from distributed import progress


def shift5(x):
    return x - 5


def inverse(x):
    return 1 / x


shifted = dask_client.map(shift5, range(1, 10))
inverted = dask_client.map(inverse, shifted)

total = dask_client.submit(sum, inverted)
display(progress(total))
total.result()

VBox()

ZeroDivisionError: division by zero

We can see which task failed:

In [14]:
[f for f in inverted if f.status == "error"]

[<Future: error, key: inverse-4f03d874c328aca27052e4758dcf2423>]

When IPython starts a worker on each engine,
it stores it in the `distributed_worker` variable in the engine's namespace.
This lets us query the worker interactively.

We can check out the current data resident on each worker:

In [15]:
%%px
dask_worker.data

Out[3:1]: Buffer<<LRU: 440/2576980377 on dict>, <Func: serialize_bytelist<->deserialize_bytes <File: /Users/minrk/dev/ip/parallel/docs/source/examples/dask-worker-space/worker-e3zdjtzn/storage, mode="a", 0 elements>>>

Now that we can poke around with each Worker,
we can have a slightly easier time figuring out what went wrong.